Requirement


1.Folder
    * './mart' 폴더에 (1)에서 저장한 Pickle 데이터가 들어 있어야 함
    * './saved_models' 폴더가 생성되어 있어야 함
2.Package
    * Python3.6
    * pandas, numpy, os, time, pickle, tensorflow, keras, matplotlib

## Data 불러오기

### Data Import from Pickle
* (1)에서 저장한 데이터 중, X_AE를 불러온다

In [2]:
import pandas as pd 
import numpy as np
import os, time, pickle
import tensorflow as tf
import keras
from keras.layers.recurrent import GRU
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import *

def write_pickle(data, path, file_name):
    with open("".join([path, '/', file_name, '.pkl']), 'wb') as f:
        pickle.dump(data, f)


def read_pickle(path, file_name):
    with open("".join([path, '/', file_name, '.pkl']), 'rb') as f:
        return pickle.load(f)

mart_path = './mart'
time_length = 44
timelength = time_length

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
X_AE = read_pickle(mart_path, '44_X_AE')
feature_cols = read_pickle(mart_path, '44_feature_cols')

In [ ]:
seq_cols = feature_cols[:time_length]
print(seq_cols)

meta_cols = feature_cols[time_length:]
print(meta_cols)

### Data Preprocessing for Sequnece Learning

In [ ]:
def preproc_for_seq(array):
    
    array_seq = []
    
    # seq_cols
    array_seq.append(array[:,:timelength].reshape((array.shape[0], timelength, 1)))
    
    # meta_cols
    array_seq.append(array[:,timelength:])
    
    return array_seq

X_AE_seq = preproc_for_seq(X_AE)

## Represenation Learning - Modeling

### Define Architecture
* ResNet Architecture를 사용
* 본래 ResNet은 Image Data에 Conv2D를 사용하지만, 본 데이터는 Sequnece Data이므로 Conv1D를 사용
* Sequence Data는 ResNet, Meta Data는 DNN
* Concatenate 후, Reconstruction함 (Autoencoder). Loss Function은 mean_squared_error
* Input -> Concatenate (Encoder)
* Concatenate -> output (Decoder)

In [ ]:
def res_unit(inputs, channels):
    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    x = Conv1D(channels, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(channels, kernel_size=3, padding='same', use_bias=False)(x)
    added = Add()([inputs, x])
    return added

def res_unit_stride(inputs, channels):
    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    x = Conv1D(channels, kernel_size=3, strides=2, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(channels, kernel_size=3, padding='same', use_bias=False)(x)
    conv = Conv1D(channels, kernel_size=1, strides=2, padding='same', use_bias=False)(inputs)
    added = Add()([conv, x])
    return added

In [ ]:
# for seq_cols, Conv1D
seq_in = Input(shape=(time_length,1))
x = Conv1D(16, kernel_size=3, activation='relu', padding='same')(seq_in)
x = MaxPooling1D(2, padding='same')(x)
x =  res_unit(x, 16)
x =  res_unit(x, 16)
x =  res_unit(x, 16)
x =  res_unit_stride(x, 32)
x =  res_unit(x, 32)
x =  res_unit(x, 32)
x =  res_unit(x, 32)
x =  res_unit_stride(x, 64)
x =  res_unit(x, 64)
x =  res_unit(x, 64)
x =  res_unit(x, 64)
x =  res_unit(x, 64)
x =  res_unit_stride(x, 128)
x =  res_unit(x, 128)
x =  res_unit(x, 128)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = GlobalAveragePooling1D()(x)
seq_out = x

# for meta_cols
meta_in = Input(shape=(len(meta_cols),))
meta_out = Dense(20, kernel_initializer='he_normal', activation='elu')(meta_in)

# Concat
merges = Concatenate()([seq_out, meta_out])

# decode
seq_decode = Dense(time_length, kernel_initializer='he_normal')(merges)
seq_decode = Reshape((time_length,1))(seq_decode)
meta_decode = Dense(len(meta_cols), kernel_initializer='he_normal')(merges)

model = Model(inputs=[seq_in, meta_in], outputs=[seq_decode, meta_decode])

In [ ]:
model.summary()

In [ ]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss=['mean_squared_error', 'mean_squared_error'], optimizer=optimizer)

### Define Callback
* Learning Rate를 잘 조절해준다는 Cyclic lr을 사용 (LRFinder Class)
* epoch = 100, batch_size = 128, train_validation split 없음

In [ ]:
%matplotlib inline
from keras.callbacks import Callback
import matplotlib.pyplot as plt
import keras.backend as K

class LRFinder(Callback):
    '''
    A simple callback for finding the optimal learning rate range for your model + dataset. 
    
    # Usage
        ```python
            lr_finder = LRFinder(min_lr=1e-5, 
                                 max_lr=1e-2, 
                                 steps_per_epoch=np.ceil(epoch_size/batch_size), 
                                 epochs=3)
            model.fit(X_train, Y_train, callbacks=[lr_finder])
            
            lr_finder.plot_loss()
        ```
    
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        epochs: Number of epochs to run experiment. Usually between 2 and 4 epochs is sufficient. 
        
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: https://arxiv.org/abs/1506.01186
    '''
    
    def __init__(self, min_lr=1e-5, max_lr=1e-2, steps_per_epoch=None, epochs=None):
        super().__init__()
        
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.total_iterations = steps_per_epoch * epochs
        self.iteration = 0
        self.history = {}
        
    def clr(self):
        '''Calculate the learning rate.'''
        x = self.iteration / self.total_iterations 
        return self.min_lr + (self.max_lr-self.min_lr) * x
        
    def on_train_begin(self, logs=None):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.min_lr)
        
    def on_batch_end(self, epoch, logs=None):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.iteration += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.iteration)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
            
        K.set_value(self.model.optimizer.lr, self.clr())
 
    def plot_lr(self):
        '''Helper function to quickly inspect the learning rate schedule.'''
        plt.plot(self.history['iterations'], self.history['lr'])
        plt.yscale('log')
        plt.xlabel('Iteration')
        plt.ylabel('Learning rate')
        
    def plot_loss(self):
        '''Helper function to quickly observe the learning rate experiment results.'''
        plt.plot(self.history['lr'], self.history['loss'])
        plt.xscale('log')
        plt.xlabel('Learning rate')
        plt.ylabel('Loss')

In [ ]:
epochs = 100
batch_size = 128

### AE Training

In [ ]:
history = model.fit(X_AE_seq, X_AE_seq,
                    batch_size=batch_size, epochs=epochs)

## Encoder Save for Finetuing in Future
* Modeling에서 Autoencoder의 중간층부터 학습에 사용하기 위해 저장한다.
* encoder도 저장하고 model(=AE)도 저장하는데, 학습에는 encoder만 사용할 예정

In [ ]:
def save_keras_model(model, filename):
    model_json = model.to_json()
    with open('{}.json'.format(filename), 'w') as json_file:
        json_file.write(model_json)
    model.save_weights('{}.h5'.format(filename))

def load_keras_model(filename):
    from keras.models import model_from_json
    json_file = open('{}.json'.format(filename), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights('{}.h5'.format(filename))
    return model

In [ ]:
model_dir = './saved_models'
filename = '{}/ResNetAE'.format(model_dir)

save_keras_model(model, filename)

In [ ]:
encoder = Model(inputs=[seq_in, meta_in], outputs=merges)

In [ ]:
filename = '{}/ResNetAE_encoder'.format(model_dir)

save_keras_model(encoder, filename)